In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM
from tensorflow.keras.models import Sequential
import importlib
import make_data as md
importlib.reload(md)
import make_model as mm
importlib.reload(mm)
import optuna
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC
from datetime import datetime as dt
from datetime import timedelta

In [2]:
data, data_np = md.load_data("東京UFJ.csv")

In [22]:
data["アッパーエンベロープー終値"] = md.upper_envelope(3, 5, 1, data_np)
data["ロワーエンベロープー終値"] = md.lower_envelope(3, 5, 1, data_np)
data["5日平均-25日平均"] = md.average_line_difference(data["5日平均"], data["25日平均"])
data["25日平均-75日平均"] = md.average_line_difference(data["25日平均"], data["75日平均"])
data_5_25 = md.cross(data["5日平均-25日平均"])
data_25_75 = md.cross(data["25日平均-75日平均"])
upper_bband, lower_bband = md.bband(data)
data["アッパーボリンジャーバンド"] = upper_bband
data["ロワーボリンジャーバンド"] = lower_bband
data["ボリンジャーバンド差"] = md.average_line_difference(data["アッパーボリンジャーバンド"] ,data["ロワーボリンジャーバンド"])
data_bband_border = md.bband_border(data["アッパーボリンジャーバンド"], data["ロワーボリンジャーバンド"], data["高値"], data["安値"])
data["MACD"] =  [x - y for (x, y) in zip(md.ema(data["終値"], 9), md.ema(data["終値"], 26))]
data["シグナルMACD"] = md.average_line(data["MACD"],9)
data_macdcross = md.cross(md.average_line_difference(data["MACD"], data["シグナルMACD"]))
data_up_or_down_2per = md.up_or_down_2per(data["終値"])
data_close=data["終値"]
data["終値対数階差変換"]=md.differ(data["終値"])

In [23]:
data = md.standardization(data)

In [24]:
data_days, data_weeks = md.data_from_year(data.index)
#data["年始からの日"] = data_days
#data["年始からの週"] = data_weeks
data['5-25クロス'] = data_5_25
data['25-75クロス'] = data_25_75
data["ボリンジャーバンドボーダー"] = data_bband_border
data["rsi9"] = md.rsi(data["終値"],9)
data["rsi14"] = md.rsi(data["終値"],14)
data["MACDクロス"] = data_macdcross

In [25]:
data = data.drop(["高値","安値","5日平均.1","25日平均.1", '5日平均', '25日平均', '75日平均','アッパーボリンジャーバンド', 'ロワーボリンジャーバンド'], axis=1)

In [26]:
data = data.fillna(0)
data_np = data.to_numpy()
data_np=np.hstack((data_np,data_up_or_down_2per))

In [37]:
seq_len = 30
epoch = 500
x_train, y_train, x_test, y_test= mm.data_split_many_class(data_np, seq_len=seq_len, y_num=3)
model = mm.build_model_many_class([data.shape[1]+3,50,100,3])

In [39]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
model.fit(x_train, y_train, batch_size=512, epochs=epoch, validation_split=0.05, callbacks=[callback])

Epoch 1/500
5/5 [==============================] - 5s 959ms/step - loss: 1.0413 - acc: 0.4395 - val_loss: 0.9507 - val_acc: 0.5238
Epoch 2/500
5/5 [==============================] - 3s 501ms/step - loss: 0.9693 - acc: 0.4807 - val_loss: 0.9361 - val_acc: 0.5159
Epoch 3/500
5/5 [==============================] - 3s 501ms/step - loss: 0.9638 - acc: 0.4908 - val_loss: 0.9394 - val_acc: 0.5000
Epoch 4/500
5/5 [==============================] - 2s 498ms/step - loss: 0.9551 - acc: 0.4912 - val_loss: 0.9438 - val_acc: 0.5079
Epoch 5/500
5/5 [==============================] - 3s 508ms/step - loss: 0.9522 - acc: 0.4950 - val_loss: 0.9423 - val_acc: 0.4921
Epoch 6/500
5/5 [==============================] - 3s 503ms/step - loss: 0.9504 - acc: 0.4887 - val_loss: 0.9421 - val_acc: 0.4683
Epoch 7/500
5/5 [==============================] - 3s 507ms/step - loss: 0.9466 - acc: 0.4987 - val_loss: 0.9352 - val_acc: 0.5238
Epoch 8/500
5/5 [==============================] - 3s 508ms/step - loss: 0.9480 - a

Epoch 58/500
5/5 [==============================] - 2s 500ms/step - loss: 0.6471 - acc: 0.7029 - val_loss: 0.8252 - val_acc: 0.6508
Epoch 59/500
5/5 [==============================] - 2s 497ms/step - loss: 0.6334 - acc: 0.7185 - val_loss: 0.8396 - val_acc: 0.6190
Epoch 60/500
5/5 [==============================] - 2s 491ms/step - loss: 0.6279 - acc: 0.7097 - val_loss: 0.8651 - val_acc: 0.6270
Epoch 61/500
5/5 [==============================] - 2s 477ms/step - loss: 0.6270 - acc: 0.7185 - val_loss: 0.8872 - val_acc: 0.6349
Epoch 62/500
5/5 [==============================] - 3s 582ms/step - loss: 0.6409 - acc: 0.6962 - val_loss: 0.8532 - val_acc: 0.6111
Epoch 63/500
5/5 [==============================] - 2s 470ms/step - loss: 0.6235 - acc: 0.7122 - val_loss: 0.8731 - val_acc: 0.5873
Epoch 64/500
5/5 [==============================] - 2s 467ms/step - loss: 0.6047 - acc: 0.7218 - val_loss: 0.9190 - val_acc: 0.6111
Epoch 65/500
5/5 [==============================] - 2s 480ms/step - loss: 0.

In [85]:
mm.evalution_many_class(x_test,y_test,model)

正解率は0.5519713261648745
混同行列[[83 45  5]
 [44 66  8]
 [ 9 14  5]]
正解率は0.5519713261648745 
適合率は[0.61029412 0.528      0.27777778] 
再現率は[0.62406015 0.55932203 0.17857143] 
F値は[0.61710037 0.54320988 0.2173913 ]


In [62]:
x_test_close=data_close[(len(data_close)-len(x_test))-1:-1]
value = mm.baibai(x_test,model,x_test_close)
print(value)

189.79999999999995


In [49]:
model.save("my_model2")
#reconstructed_model = tf.keras.models.load_model("my_model")
#mm.draw_tran_result(reconstructed_model, x_test, y_test)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: my_model2\assets


In [17]:
x_train_sfs=data.iloc[:,:-1]
y_train_sfs=data.iloc[:,-1]

sfs1 = SFS(RandomForestClassifier(), # 使う学習器
           k_features=10, #特徴をいくつまで選択するか
           forward=True, #Trueでforward selectionになる。Falseでback
           floating=False, #後に説明するが、forward selectionの亜種を行うためのもの。
           verbose=2, #実行時のlogをどれだけ詳しく表示するか
           scoring='accuracy', # 評価指標
           cv=5) #クロスバリデーション
#ちなみにクロスバリデーションをしない(cv=0)とすると、最終的に選ばれる特徴が変わります。

sfs1 = sfs1.fit(x_train_sfs,y_train_sfs)

selected_feat = x_train_sfs.columns[list(sfs1.k_feature_idx_)]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed:  1.1min finished

[2022-06-09 02:12:17] Features: 1/10 -- score: 0.5584583439815998[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   55.6s finished

[2022-06-09 02:13:12] Features: 2/10 -- score: 0.6070783286480961[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   50.8s finished

[2022-06-09 02:14:03] Features: 3/10 -- score: 0.607794531050345[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   

In [18]:
sfs1.subsets_

{1: {'feature_idx': (25,),
  'cv_scores': array([0.5625    , 0.54642857, 0.55813953, 0.5509839 , 0.57423971]),
  'avg_score': 0.5584583439815998,
  'feature_names': ('MACDクロス',)},
 2: {'feature_idx': (22, 25),
  'cv_scores': array([0.6125    , 0.60357143, 0.60465116, 0.59928444, 0.61538462]),
  'avg_score': 0.6070783286480961,
  'feature_names': ('ボリンジャーバンドボーダー', 'MACDクロス')},
 3: {'feature_idx': (20, 22, 25),
  'cv_scores': array([0.60357143, 0.61071429, 0.60107335, 0.60644007, 0.61717352]),
  'avg_score': 0.607794531050345,
  'feature_names': ('5-25クロス', 'ボリンジャーバンドボーダー', 'MACDクロス')},
 4: {'feature_idx': (20, 21, 22, 25),
  'cv_scores': array([0.61607143, 0.59285714, 0.59570662, 0.61001789, 0.61359571]),
  'avg_score': 0.6056497572195247,
  'feature_names': ('5-25クロス', '25-75クロス', 'ボリンジャーバンドボーダー', 'MACDクロス')},
 5: {'feature_idx': (11, 20, 21, 22, 25),
  'cv_scores': array([0.60178571, 0.59107143, 0.56887299, 0.62254025, 0.57423971]),
  'avg_score': 0.5917020189113212,
  'feature_names'